In [31]:
# Kütüphaneler eklendi
import pandas as pd
import numpy as np
import os  # Klasör işleri için lazım
import joblib  # Modeli kaydetmek için lazım
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score

In [32]:
# Temizlenmiş veri seti okuunması
df = pd.read_csv('../data/processed/main_salary_dataset.csv')
print("Veri seti yüklendi")


Veri seti yüklendi


In [33]:
#Çözümleme
education_map = {
    "unknown":0,
    "high_school":1,
    "bachelor":2,
    "master":3,
    "doctor":4,
}
df['education_level_encoded'] = df['education_level'].map(education_map)
seniority_map = {
    'junior': 0,
    'senior': 1
}
df['seniority_level_encoded'] = df['seniority_level'].map(seniority_map)

In [34]:
le_job = LabelEncoder()
df['job_title_encoded'] = le_job.fit_transform(df['job_title'])

In [35]:
# X: Maaşı tahmin etmek için kullanacağımız bilgiler
# y: Hedefimiz (Maaş)
X = df[['age', 'years_experience', 'education_level_encoded', 'seniority_level_encoded', 'job_title_encoded']]
y = df['salary']

In [36]:
# Veriyi %80 Eğitim, %20 Test diye ikiye bölünmesi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
# Model eğitimi (RANDOM FOREST)
print("Model eğitimine geçildi")
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
print("Eğitim tamamlandı")

Model eğitimine geçildi
Eğitim tamamlandı


In [38]:
# Test verileri ile modelin performansını kontrol edilimi
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [39]:
# Test sonuçları 
print(f"\n--- SONUÇLAR ---")
print(f"Ortalama Hata Payı (MAE): {mae:.2f}")
print(f"Başarı Skoru (R2): {r2:.4f}")


--- SONUÇLAR ---
Ortalama Hata Payı (MAE): 12560.64
Başarı Skoru (R2): 0.8737


In [40]:
#modelin kayıt yolu 
models_dir = '../models'
model_path = os.path.join(models_dir, 'salary_model_random_forest.pkl')
encoder_path = os.path.join(models_dir, 'job_title_encoder.pkl')

In [42]:
#Kayıt 
joblib.dump(model, model_path)
joblib.dump(le_job, encoder_path)

print(f"\n Model kaydedildi.")
print(f"Model şuraya kaydedildi: {model_path}")
print(f"Encoder şuraya kaydedildi: {encoder_path}")


 Model kaydedildi.
Model şuraya kaydedildi: ../models\salary_model_random_forest.pkl
Encoder şuraya kaydedildi: ../models\job_title_encoder.pkl
